In [1]:
# General imports ..
import os
import random
import logging
import urllib3
import warnings
import json
import requests

# langchain imports ..
from langchain.retrievers import WikipediaRetriever
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from urls import base_url, api_url, get_sub_url, get_sub_chap_url, add_ai_url
from logger import setup_logger

# surpress warnings ..
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langchain")

# Setting up logger ..
try:
    # Create 'logs' directory if it doesn't exist
    logs_directory = 'logs'
    if not os.path.exists(logs_directory):
        os.makedirs(logs_directory)

    # Set up logger with the log file inside the 'logs' directory
    log_file_path = os.path.join(logs_directory, 'add_chap_topics.log')
    logger = setup_logger('get_topics', log_file_path)
    logger.info('Logger is activated for GENERATING Chapter & topics ..')
except Exception as e:
    # Log the exception if there is an error during logger setup
    logger.error(e)

# Languages list ..
lang_list = ["english", "hindi", "bengali", "marathi", "telugu", "tamil", "gujarati", "urdu", "kannada", "malayalam", "oriya", "punjabi", "assamese", "sinhala"]
lang_codes = {'english': 1, 'hindi': 6}

# Setting OpenAi key ..
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
openai_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

# surpress warnings ..
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langchain")

In [2]:
# 1. Getting Subjects list from database ..  
def get_subjects():
    try:
        subject_response = requests.get(get_sub_url, verify=False)
        if subject_response.status_code==200:
            #logger.info(f"GET Request for added Subjects list successfull with status code {subject_response.status_code}")
            #print("Subjects list has been retrieved successfully .. ")
            
            response_str = subject_response.content.decode('utf-8')
            subjects = json.loads(response_str)['data']
            return subjects
        else:
            print(f"GET Request failed with status code {subject_response.status_code}")
    except Exception as e:
        logger.error("GET Subjects API is down .. ")

# 2.  Getting Subject info ..
def get_subject_info(subject_name, subject_list=get_subjects()):
    for subject in subject_list:
        if subject['name'] == subject_name:
            return (subject['_id'], subject['name'])

# 3.  Getting chapters list ..
def get_chapters(subject):
    subject_name = subject[1]
    print(f"Processing subject - {subject_name}")
        
    try:
        #print('Triggering GET Chapters List API ..')
        url = get_sub_chap_url + subject[0]
        chapters_response = requests.get(url, verify=False)
        if chapters_response.status_code==200:        
            response_str = chapters_response.content.decode('utf-8')
            chapters = json.loads(response_str)['data']
            return chapters
        else:
            print(f"GET Request for subject - {subject_name} failed with status code {chapters_response.status_code}")

    except Exception as e:
        print("GET Chapters API is down .. ")

# Getting grades by Grade Id ..
def find_grade_by_id(grade_id):
    grades_data = {
        "6": "80c7a75f-3060-11ee-bb86-000c29539bc1",
        "7": "92673410-3060-11ee-bb86-000c29539bc1",
        "8": "a17f9d23-3060-11ee-bb86-000c29539bc1",
        "9": "b2f6496b-3060-11ee-bb86-000c29539bc1",
        "10": "c114adfe-3060-11ee-bb86-000c29539bc1",
        "11 (Science)": "d64aca1f-3060-11ee-bb86-000c29539bc1",
        "11 (Commerce)": "e8404d03-3060-11ee-bb86-000c29539bc1",
        "11 (Arts)": "f971156a-3060-11ee-bb86-000c29539bc1",
        "12 (Science)": "2a79e82a-3061-11ee-bb86-000c29539bc1",
        "12 (Commerce)": "55acd452-3061-11ee-bb86-000c29539bc1",
        "12 (Arts)": "81dbdcdb-3061-11ee-bb86-000c29539bc1"
    }
    # matching with IDs ..
    for key, value in grades_data.items():
        if value == grade_id:
            return key
            
# Getting topics input_dict  ..
def get_topic_dicts(subject):
    dicts = []
    chapters = get_chapters(subject)
    for chapter in enumerate(chapters):
        count = chapter[0]
        chapterName = chapter[1]['name']
        
        # Getting Topics dict ..
        for item in enumerate(chapter[1]['topics']):
             topic_count = item[0]
             grade_id = item[1]['cur_tag'][0]['grd_id']
             topic_grade = find_grade_by_id(grade_id)
             data_dict = {"subject":subject[1], "chapter_count":count, "chapterName":chapterName, "topic_id":item[1]['_id'], "topicName": item[1]['name'], "topic_grade":topic_grade}
             dicts.append(data_dict)
    return dicts

# 4. Generate answers ..
def gen_content(prompt):
    model_name = "gpt-4-1106-preview"
    temperature = 0.5
    model = OpenAI(model_name=model_name, temperature=temperature)
    output = model(prompt)
    return output

languages_dict = {
    "Afrikaans": "af", "Albanian": "sq", "Amharic": "am", "Arabic": "ar", "Armenian": "hy", "Assamese": "as",
    "Aymara": "ay", "Azerbaijani": "az", "Bambara": "bm", "Basque": "eu", "Belarusian": "be", "Bengali": "bn",
    "Bhojpuri": "bho", "Bosnian": "bs", "Bulgarian": "bg", "Catalan": "ca", "Cebuano": "ceb",
    "Chinese (Simplified)": "zh-CN", "Chinese (Traditional)": "zh-TW", "Corsican": "co", "Croatian": "hr",
    "Czech": "cs", "Danish": "da", "Dhivehi": "dv", "Dogri": "doi", "Dutch": "nl", "English": "en",
    "Esperanto": "eo", "Estonian": "et", "Ewe": "ee", "Filipino (Tagalog)": "fil", "Finnish": "fi",
    "French": "fr", "Frisian": "fy", "Galician": "gl", "Georgian": "ka", "German": "de", "Greek": "el",
    "Guarani": "gn", "Gujarati": "gu", "Haitian Creole": "ht", "Hausa": "ha", "Hawaiian": "haw",
    "Hebrew": "he", "Hindi": "hi", "Hmong": "hmn", "Hungarian": "hu", "Icelandic": "is", "Igbo": "ig",
    "Ilocano": "ilo", "Indonesian": "id", "Irish": "ga", "Italian": "it", "Japanese": "ja", "Javanese": "jv",
    "Kannada": "kn", "Kazakh": "kk", "Khmer": "km", "Kinyarwanda": "rw", "Konkani": "gom", "Korean": "ko",
    "Krio": "kri", "Kurdish": "ku", "Kurdish (Sorani)": "ckb", "Kyrgyz": "ky", "Lao": "lo", "Latin": "la",
    "Latvian": "lv", "Lingala": "ln", "Lithuanian": "lt", "Luganda": "lg", "Luxembourgish": "lb",
    "Macedonian": "mk", "Maithili": "mai", "Malagasy": "mg", "Malay": "ms", "Malayalam": "ml", "Maltese": "mt",
    "Maori": "mi", "Marathi": "mr", "Meiteilon (Manipuri)": "mni-Mtei", "Mizo": "lus", "Mongolian": "mn",
    "Myanmar (Burmese)": "my", "Nepali": "ne", "Norwegian": "no", "Nyanja (Chichewa)": "ny", "Odia (Oriya)": "or",
    "Oromo": "om", "Pashto": "ps", "Persian": "fa", "Polish": "pl", "Portuguese (Portugal, Brazil)": "pt",
    "Punjabi": "pa", "Quechua": "qu", "Romanian": "ro", "Russian": "ru", "Samoan": "sm", "Sanskrit": "sa",
    "Scots Gaelic": "gd", "Sepedi": "nso", "Serbian": "sr", "Sesotho": "st", "Shona": "sn", "Sindhi": "sd",
    "Sinhala (Sinhalese)": "si", "Slovak": "sk", "Slovenian": "sl", "Somali": "so", "Spanish": "es",
    "Sundanese": "su", "Swahili": "sw", "Swedish": "sv", "Tagalog (Filipino)": "tl", "Tajik": "tg", "Tamil": "ta",
    "Tatar": "tt", "Telugu": "te", "Thai": "th", "Tigrinya": "ti", "Tsonga": "ts", "Turkish": "tr", "Turkmen": "tk",
    "Twi (Akan)": "ak", "Ukrainian": "uk", "Urdu": "ur", "Uyghur": "ug", "Uzbek": "uz", "Vietnamese": "vi",
    "Welsh": "cy", "Xhosa": "xh", "Yiddish": "yi", "Yoruba": "yo", "Zulu": "zu"
}

# getting Language codess ..
def get_language_code(language, languages=languages_dict):
    for lang, code in languages.items():
        if lang.lower() == language.lower():
            return code
            
# Translating ..
from googletrans import Translator
translator = Translator()

def translate(text, lang):
    translated_text = translator.translate(text, dest=get_language_code(lang))
    return translated_text.text

In [3]:
def gen_text_prompts(topic, subject, chapter, grade, target_language="English"):
    prompts = []
    level_instructions = {
        1: "For Level 1 students, provide a simple and detailed explanation.",
        2: "For Level 2 students, provide a balanced and detailed explanation.",
        3: "For Level 3 students, provide a detailed explanation with increased difficulty.",
        4: "For Level 4 students, provide a concise and challenging explanation."
    }

    for level, instruction in level_instructions.items():
        instructions = (
            f"Generate the explanation in {target_language}.\n\n"
            f"Provide a comprehensive and detailed explanation or answer in {target_language}. "
            "Consider the following structure:\n\n"
            f"{topic} Defined:\n"
            f"Start by defining the topic '{topic}' in detail. Explain any concepts, mathematical relationships, or relevant information related to {subject}.\n\n"
            f"Insert any additional content specific to the topic here.\n"
            f"Examples:\n"
            f"Include sample examples to illustrate key points or demonstrate the procedure.\n"
            f"Insert example content here.\n"
            f"Important Points and Notes:\n"
            f"Highlight any crucial information or notable aspects related to the topic or procedure.\n"
            f"Insert important points and notes here.\n"
            f"Additional Information:\n"
            f"Add any other relevant information, historical context, or interesting facts to enhance the answer.\n\n"
            f"Topic: {topic}\n"
            "When generating the answer, follow the structure provided."
        )
        prompts.append(instructions)
    return prompts

# main fn.  ..
def get_content(subject, lang):
    topics = get_topic_dicts(get_subject_info(subject))
    prompts = gen_text_prompts(topics[0]['topicName'], subject=topics[0]['subject'], chapter=topics[0]['chapterName'], 
                               grade=topics[0]['topic_grade'], target_language="english")
    out = gen_content(prompts[0])
    out = translate(out, lang)
    return out

out = get_content('Mathematics', lang='hindi')
print(out)

Processing subject - Mathematics
परिभाषित संख्याओं का उद्देश्य:
संख्याओं का उद्देश्य गणित और दैनिक जीवन में एक मौलिक अवधारणा है। संख्याएँ अमूर्त प्रतीक हैं जो मात्राओं का प्रतिनिधित्व करते हैं, जो हमें हमारे आस-पास की दुनिया को मापने, गणना करने और समझने की अनुमति देते हैं। वे गणित के निर्माण खंड हैं, जो वस्तुओं और अवधारणाओं की गिनती, क्रम, लेबलिंग और मात्रा निर्धारित करने के लिए आवश्यक हैं। संख्याएँ हमें जोड़, घटाव, गुणा और भाग जैसे अंकगणितीय ऑपरेशन करने में सक्षम बनाती हैं, जो अधिक जटिल गणितीय सिद्धांतों और अनुप्रयोगों का आधार हैं।

गणित में, संख्याओं को विभिन्न प्रकारों में वर्गीकृत किया जाता है, जैसे प्राकृतिक संख्याएँ, पूर्णांक, तर्कसंगत संख्याएँ और वास्तविक संख्याएँ, जिनमें से प्रत्येक का एक विशिष्ट उद्देश्य होता है। प्राकृतिक संख्याओं (1, 2, 3, ...) का उपयोग गिनती और क्रम देने के लिए किया जाता है। पूर्णांक (... -3, -2, -1, 0, 1, 2, 3, ...) में नकारात्मक संख्याएं शामिल होती हैं और उन परिदृश्यों में उपयोग की जाती हैं जिनमें दिशा या संतुलन शामिल होता है, जैसे वित्तीय लेनदेन। परिमेय 

In [20]:
import re

def embed_html_tags(paragraph):
    # Define HTML instructions
    html_instructions = {
        "heading": "<h{level}>",
        "paragraph": "<p>",
        "strong": "<strong>",
        "subscript": "<sub>",
        "superscript": "<sup>",
        "mathjax_inline": "$$",
        "mathjax_block": "$$"
    }

    # Construct the prompt with instructions, paragraph, and HTML format instructions
    prompt = (
        f"{html_instructions['paragraph']}Please embed HTML tags as necessary to format the following paragraph:\n\n"
        f"{paragraph}\n\n"
        "You may use the following HTML tags:\n"
        "- <h{{level}}> for headings\n"
        "- <p> for paragraphs\n"
        "- <strong> for bold text\n"
        "- <sub> for subscript\n"
        "- <sup> for superscript\n"
        "- $$ for inline MathJax\n"
        "- $$ for block MathJax\n"
        "- <li> for list items\n"
        "Feel free to use additional HTML tags as needed to enhance the structure of the text."
    )

    return prompt

# Driver code ..
prompt = embed_html_tags(out)
out = gen_content(prompt)
print(out)

```html
<h1>परिभाषित संख्याओं का उद्देश्य:</h1>
<p><strong>संख्याएँ मौलिक इकाइयाँ हैं</strong> जिनका उपयोग गणित और रोजमर्रा की जिंदगी में गिनती, मापने, लेबलिंग और कोडिंग के लिए किया जाता है। वे अमूर्त अवधारणाएँ हैं जो मात्राओं का प्रतिनिधित्व करती हैं और हमें मात्रात्मक तरीके से हमारे आसपास की दुनिया को समझने और उसमें हेरफेर करने की अनुमति देती हैं। गणित में, संख्याओं का उपयोग संचालन करने, समीकरणों को हल करने और वास्तविक दुनिया की घटनाओं का वर्णन करने वाले मॉडल बनाने के लिए किया जाता है।</p>

<p>संख्याओं को <strong>विभिन्न प्रकारों में वर्गीकृत</strong> किया जा सकता है, जैसे प्राकृतिक संख्याएँ, पूर्ण संख्याएँ, पूर्णांक, परिमेय संख्याएँ, अपरिमेय संख्याएँ और सम्मिश्र संख्याएँ। प्रत्येक श्रेणी में विशिष्ट गुण और उपयोग होते हैं। गिनती के लिए प्राकृतिक संख्याओं (1, 2, 3, ...) का उपयोग किया जाता है; पूर्ण संख्याओं में सभी प्राकृतिक संख्याएँ और शून्य शामिल हैं; पूर्णांक ऋणात्मक संख्याओं के साथ पूर्ण संख्याओं का विस्तार करते हैं; परिमेय संख्याएँ पूर्णांकों के भागफल को दर्शाने वाली भिन्न हैं; अ

In [22]:
prompt

'<p>Please embed HTML tags as necessary to format the following paragraph:\n\nपरिभाषित संख्याओं का उद्देश्य:\nसंख्याएँ मौलिक इकाइयाँ हैं जिनका उपयोग गणित और रोजमर्रा की जिंदगी में गिनती, मापने, लेबलिंग और कोडिंग के लिए किया जाता है। वे अमूर्त अवधारणाएँ हैं जो मात्राओं का प्रतिनिधित्व करती हैं और हमें मात्रात्मक तरीके से हमारे आसपास की दुनिया को समझने और उसमें हेरफेर करने की अनुमति देती हैं। गणित में, संख्याओं का उपयोग संचालन करने, समीकरणों को हल करने और वास्तविक दुनिया की घटनाओं का वर्णन करने वाले मॉडल बनाने के लिए किया जाता है।\n\nसंख्याओं को विभिन्न प्रकारों में वर्गीकृत किया जा सकता है, जैसे प्राकृतिक संख्याएँ, पूर्ण संख्याएँ, पूर्णांक, परिमेय संख्याएँ, अपरिमेय संख्याएँ और सम्मिश्र संख्याएँ। प्रत्येक श्रेणी में विशिष्ट गुण और उपयोग होते हैं। गिनती के लिए प्राकृतिक संख्याओं (1, 2, 3, ...) का उपयोग किया जाता है; पूर्ण संख्याओं में सभी प्राकृतिक संख्याएँ और शून्य शामिल हैं; पूर्णांक ऋणात्मक संख्याओं के साथ पूर्ण संख्याओं का विस्तार करते हैं; परिमेय संख्याएँ पूर्णांकों के भागफल को दर्शा